In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

NPM= 75
np.random.seed(NPM)

makan =  pd.read_csv("makanan.csv")
# Fill NaN values in 'harga' column with 0
makan['harga'] = makan['harga'].fillna(0)
# mohon ditambahkan (lagi) agar lebih beragam

budget= 1e6 #rupiah

wkenyang_base= 0.8
wrasa_base= 0.7
wsehat_base= 0.9 # simpangan lebih besar
wbosan_base= 0.2

skor_kumulatif= 0.0
SAMPLE_NUM= 500

sampleid= np.arange(0, 500)
# skor perorangan
skor= np.zeros(SAMPLE_NUM, dtype=np.double)
sisa= np.zeros(SAMPLE_NUM, dtype=np.double)
sehat= np.zeros(SAMPLE_NUM, dtype=np.double)
pilihankum= np.zeros(SAMPLE_NUM, dtype=np.double)

SISA_MININUM= 150e3
SISA_MAKSIMUM= 300e3
bin_sisa= np.arange(SISA_MININUM, SISA_MAKSIMUM, 1000)
resp_sehat= np.zeros((SAMPLE_NUM, len(bin_sisa)), dtype=np.double)
resp_pilih= np.zeros((SAMPLE_NUM, 62), dtype=np.double) # Changed to 62 to accommodate 2 selections per day for 31 days

for n in range(0, SAMPLE_NUM):
    budget=1e6
    skor_kumulatif= 0
    skor_sehat= 0

    for i in range(0, 31): # hari
        # makan pertama, pilih secara acak
        pilihan= np.random.randint(0, len(makan))
        # update weight factor
        w_kenyang= np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa= np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat= np.random.normal(loc=wsehat_base, scale=0.4)
        w_bosan= np.random.normal(loc=wbosan_base, scale=0.2)
        skor_kumulatif= skor_kumulatif + \
            w_kenyang * makan['kenyang'][pilihan]\
            + w_rasa * makan['rasa'][pilihan] \
            + w_sehat * makan['sehat'][pilihan] \
            - w_bosan * makan['bosan'][pilihan]
        skor_sehat += makan['sehat'][pilihan] - 0.5
        budget = budget - makan['harga'][pilihan]
        #print(f"hari {i} pagi: {makan['makanan'][pilihan]}")
        resp_pilih[n][2*i]= pilihan # Changed index to 2*i

        # makan keduas
        pilihan= np.random.randint(0, len(makan))
        # update weight factor
        w_kenyang= np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa= np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat= np.random.normal(loc=wsehat_base+0.2, scale=0.4)
        w_bosan= np.random.normal(loc=wbosan_base*2, scale=0.2)
        skor_kumulatif= skor_kumulatif + \
            w_kenyang * makan['kenyang'][pilihan]\
            + w_rasa * makan['rasa'][pilihan] \
            + w_sehat * makan['sehat'][pilihan] \
            - w_bosan * makan['bosan'][pilihan]
        skor_sehat += makan['sehat'][pilihan] - 0.5
        budget = budget - makan['harga'][pilihan]
        #print(f"hari {i} malam: {makan['makanan'][pilihan]}")
        resp_pilih[n][2*i+1]= pilihan # Changed index to 2*i+1

    # print(f"skor: {skor_kumulatif}, sisa uang: {budget}")
    sisa[n]= budget
    skor[n]= skor_kumulatif
    sehat[n]= skor_sehat
    # print(f"{n} skor sehat:{skor_sehat}")

    # Check if budget is within the desired range before calculating the index
    if SISA_MININUM <= budget < SISA_MAKSIMUM:
        resp_sehat[n][ int((budget-SISA_MININUM)/1000)]= skor_sehat


# np.corrcoef(skor, sisa)
# np.histogram(sisa)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
Y, X = np.meshgrid(bin_sisa, sampleid)
# Colorize the surface using a colormap
surf = ax.plot_surface(X, Y, resp_sehat,
                       cmap='jet_r',   # try 'plasma', 'coolwarm', etc.
                       edgecolor='none')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
# plot yang ini barangkali butuh dipercantik

# TUGAS
# (1) cari pola makan mana saja yang menghasilkan nilai kumulatif kesehatan (resp_sehat) positif
# (2) cari titik-titik dengan sisa uang bulanan positif dari (1)
# (3) cari titik dengan skor skumulatif terbesar (skor) di mana nilai kumulatif kesehatannya positif
# (4) tampikan jadwal makanan sesuai pilihan (resp_pilih)

In [ ]:
# cari pola makan mana saja yang menghasilkan nilai kumulatif kesehatan (resp_sehat) positif
indeks_positif = np.where(sehat > 0)[0]

print("Jumlah pola makan sehat:", len(indeks_positif))
print("Contoh indeks:", indeks_positif[:5])

for idx in indeks_positif[:5]:
    print(f"\nSampel {idx} (Skor sehat: {sehat[idx]:.2f})")
    pilihan_harian = resp_pilih[idx]
    for hari in range(31):
        pagi = int(pilihan_harian[2*hari])
        malam = int(pilihan_harian[2*hari+1])
        print(f"Hari {hari+1}: {makan['makanan'][pagi]} (pagi), {makan['makanan'][malam]} (malam)")


In [ ]:
#cari titik-titik dengan sisa uang bulanan positif dari (1)
indeks_sehat_sisa = np.where((sehat > 0) & (sisa > 0))[0]

print("Jumlah pola makan sehat + sisa positif:", len(indeks_sehat_sisa))
print("Contoh indeks:", indeks_sehat_sisa[:5])

for idx in indeks_sehat_sisa[:5]:
    print(f"\nSampel {idx} (Skor sehat: {sehat[idx]:.2f}, Sisa uang: {sisa[idx]:.0f})")
    pilihan_harian = resp_pilih[idx]
    for hari in range(31):
        pagi = int(pilihan_harian[2*hari])
        malam = int(pilihan_harian[2*hari+1])
        print(f"Hari {hari+1}: {makan['makanan'][pagi]} (pagi), {makan['makanan'][malam]} (malam)")


In [ ]:
#cari titik dengan skor skumulatif terbesar (skor) di mana nilai kumulatif kesehatannya positif
indeks_sehat = np.where(sehat > 0)[0]

if len(indeks_sehat) > 0:
    idx_maks = indeks_sehat[np.argmax(skor[indeks_sehat])]
    print(f"Titik dengan skor kumulatif terbesar: Sampel {idx_maks}")
    print(f"Skor: {skor[idx_maks]:.2f}, Skor Sehat: {sehat[idx_maks]:.2f}, Sisa Uang: {sisa[idx_maks]:.0f}")

    pilihan_harian = resp_pilih[idx_maks]
    for hari in range(31):
        pagi = int(pilihan_harian[2*hari])
        malam = int(pilihan_harian[2*hari+1])
        print(f"Hari {hari+1}: {makan['makanan'][pagi]} (pagi), {makan['makanan'][malam]} (malam)")
else:
    print("Tidak ada sampel dengan kesehatan positif.")


In [ ]:
# (4) tampikan jadwal makanan sesuai pilihan (resp_pilih)
import pandas as pd

idx_jadwal = idx_maks
pilihan_harian = resp_pilih[idx_jadwal]

jadwal = []
for hari in range(31):
    pagi = int(pilihan_harian[2*hari])
    malam = int(pilihan_harian[2*hari+1])
    jadwal.append({
        "Hari": hari+1,
        "Pagi": makan['makanan'][pagi],
        "Malam": makan['makanan'][malam]
    })

df_jadwal = pd.DataFrame(jadwal)
df_jadwal
